In [3]:
import MMM_010519 as MMM
import warnings
import pandas as pd
import numpy as np
import os


# df_LL_all_models.to_pickle("data/010519/temp_results")



Load word per topic probablity matrices  as outputted by the models
Choose the model of intrest to regenerate data
Take into consideration that pi is generated randomly so data can not be fully restored

In [24]:

#MMM and LDA output, toics 12/15, 


MMM_12_E = pd.read_csv("data/input_data/MMM_12_e.csv", index_col=0).values
LDA_12_E = pd.read_csv("data/input_data/LDA_12_e.csv", index_col=0).values
LDA_15_E = pd.read_csv("data/input_data/LDA_15_e.csv", index_col=0).values


#CTM output, 12/15 topics, data is outputted in log format, needs exp to be used
CTM_12_E = pd.read_csv("data/input_data/CTM_12_e.csv", index_col=0).reset_index().values
CTM_12_E = np.exp(CTM_12_E)

CTM_15_E = pd.read_csv("data/input_data/CTM_15_e.csv", index_col=0).reset_index().values
CTM_15_E = np.exp(CTM_15_E)


#Mutation counts matrix
X = pd.read_csv("data/input_data/mutation_counts.csv",index_col=0) #Mutation counts matrix

#Original data
data = pd.read_json("/Users/dror/PycharmProjects/MMM workshop/data/ICGC-BRCA.json")



#Ignore warnings caused by non existant mutations.
warnings.filterwarnings("ignore", message="divide by zero encountered in log")

n = 10 #num of repitiions per each cross-validation, this is the length of the likelihood vector

#Generate list of chromosomes for cross validation
chrom_lst = list(range(1,23))
chrom_lst.extend('X')


In [5]:

#Create a dictionary of LL lists for each chromosome, later to be converted to a DataFrame
ll_dict = {} 
model_name = 'mmm' #Change to whaterver model you are testing
MMM.num_of_topics = 12 #Change to desired number of topics
n = 10 #change number of desired iterations

#Iterate Through chromosomes, preform cross validation for each chromosomes.
for chrom_id in chrom_lst:
    #held out chromosme will later use for testing
    test = pd.read_csv(f"data/cross_val_datasets/chrom{chrom_id}.csv").drop("Unnamed: 0", axis = 1) 
    #dataset with a single chromosome out
    training_set = pd.read_csv(f"data/cross_val_datasets/full_data_chrom{chrom_id}out.csv").drop("Unnamed: 0", axis = 1) 
    
    
    print(f"Now training MMM model, on full dataset with chrom #{chrom_id} out.\n")
    ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'] =  []
    
    #For each chromosome generate 10 results to overcome random data
    for i in range(n):
        print(f"Iteration: #{i} : ", end = "")        
        Model = MMM.fit(training_set, 0.001, 50, MMM_12_E) #Change input matrix to whichever matrix you want
        if(MMM == 1): #if model didnot converge, 1 is appended instead of a negative likelihood score
            print("Did not converrge, appending 1\n")
            ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'].append(1.0)
        else:
            #Else model converged, check LL and append it.
            print("Model converged, appending LL score: ", end = "")
            ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'].append(MMM.Theta.log_likelihood2(test, Model))
            print(ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'][i])
            
        print(f"Finished Iteration: {i}\n")
        


Now training MMM model, on full dataset with chrom #1 out.

Iteration: 0 - Converged after 2 iterations
Model converged, appending LL score: -77.48026571698358
Finished Iteration: 0

Iteration: 1 - 

KeyboardInterrupt: 

Generate a DataFrame from the dictionary. each column is a list of cross validation results. 
Save Data to csv for further use.

In [8]:
display(ll_dict)
df_mmm = pd.DataFrame(ll_dict)
df_mmm.to_csv("data/final_data/mmm_crossval_results_12_unsummed.csv")

In [12]:
df_mmm.insert(loc = 0, column = 'score', value = df_mmm_12.sum(axis=1))
df_final = df_mmm.iloc[:,0:1] #Create final DataFrame containing only the summed likelihood score 



Add additional data to DataFrame represeting number of topics, type of model etc.

In [14]:
df_final['model'] = 'mmm'
df_final['iter'] = mmm_12_final.index
df_final['topics'] = 12


/Users/dror/.conda/envs/MMM workshop/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/dror/.conda/envs/MMM workshop/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/dror/.conda/envs/MMM workshop/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [15]:
df_final

,score,model,iter,topics
0,-1049.385205,mmm,0,12
1,-1049.690610,mmm,1,12
2,-1049.400285,mmm,2,12
3,-1049.016754,mmm,3,12
4,-1049.634612,mmm,4,12
5,-1049.147541,mmm,5,12
6,-1049.309411,mmm,6,12
7,-1049.669893,mmm,7,12
8,-1049.053046,mmm,8,12
9,-1049.531606,mmm,9,12


In [17]:
df_final.to_csv(f"data/final_data/{model_name}_crossval_results_{MMM.num_of_topics}.csv")
df_final.to_pickle(f"data/final_data/{model_name}_crossval_results_{MMM.num_of_topics}.pkl")

Learn word per topic probablity matrice for the MMM model with 14 topics.

In [3]:
MMM.num_of_topics = 14
MMM.num_of_topics

14

In [5]:
Model = MMM.fit(X, 0.001, 100)

Converged after 2 iterations


In [19]:
word_probs_mat = Model[0].e
word_probs_mat = np.exp(word_probs_mat)
MMM_14_E = pd.DataFrame(word_probs_mat)
type(MMM_14_E)

pandas.core.frame.DataFrame

In [5]:
MMM_14_E = pd.read_csv(f"data/080519/MMM_14_term_matrix.csv",  index_col=0).values


In [6]:
def generate_LL_df(model_name : str, num_of_topics :int, n: int, E_mat: pd.DataFrame):
    
    #Create a dictionary of LL lists for each chromosome, later to be converted to a DataFrame
    ll_dict = {} 
    model_name = model_name #Change to whaterver model you are testing
    MMM.num_of_topics = num_of_topics #Change to desired number of topics
    n = 10 #change number of desired iterations
    
    
    #Iterate Through chromosomes, preform cross validation for each chromosomes.
    for chrom_id in chrom_lst:
        #held out chromosme will later use for testing
        test = pd.read_csv(f"data/cross_val_datasets/chrom{chrom_id}.csv").drop("Unnamed: 0", axis = 1) 
        #dataset with a single chromosome out
        training_set = pd.read_csv(f"data/cross_val_datasets/full_data_chrom{chrom_id}out.csv").drop("Unnamed: 0", axis = 1) 


        print(f"Now training MMM model, on full dataset with chrom #{chrom_id} out.\n")
        ll_dict[f'{model_name}_{num_of_topics}_{chrom_id}'] =  []

        #For each chromosome generate 10 results to overcome random data
        for i in range(n):
            print(f"Iteration: #{i} : ", end = "")        
            Model = MMM.fit(training_set, 0.001, 50, E_mat) #Change input matrix to whichever matrix you want
            if(MMM == 1): #if model didnot converge, 1 is appended instead of a negative likelihood score
                print("Did not converrge, appending 1\n")
                ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'].append(1.0)
            else:
                #Else model converged, check LL and append it.
                print("Model converged, appending LL score: ", end = "")
                ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'].append(MMM.Theta.log_likelihood2(test, Model))
                print(ll_dict[f'{model_name}_{MMM.num_of_topics}_{chrom_id}'][i])

            print(f"Finished Iteration: {i}\n")
    
    return pd.DataFrame(ll_dict)


In [6]:
df = MMM.generate_LL_df('mmm', 14,MMM_14_E, chrom_lst)

Num of topics is : 14
Now training MMM model, on full dataset with chrom #1 out.

Iteration: #0 : Converged after 2 iterations
Model converged, appending LL score: -82.76667037944506
Finished Iteration: 0

Iteration: #1 : Converged after 2 iterations
Model converged, appending LL score: -82.76483736460256
Finished Iteration: 1

Iteration: #2 : Converged after 2 iterations
Model converged, appending LL score: -82.78332118527882
Finished Iteration: 2

Iteration: #3 : Converged after 2 iterations
Model converged, appending LL score: -82.77486797523163
Finished Iteration: 3

Iteration: #4 : Converged after 2 iterations
Model converged, appending LL score: -82.76397126406883
Finished Iteration: 4

Iteration: #5 : Converged after 2 iterations
Model converged, appending LL score: -82.78011856180079
Finished Iteration: 5

Iteration: #6 : Converged after 2 iterations
Model converged, appending LL score: -82.7925063667085
Finished Iteration: 6

Iteration: #7 : Converged after 2 iterations
Model 

Iteration: #3 : Converged after 2 iterations
Model converged, appending LL score: -69.36515636043514
Finished Iteration: 3

Iteration: #4 : Converged after 2 iterations
Model converged, appending LL score: -69.3661609794818
Finished Iteration: 4

Iteration: #5 : Converged after 2 iterations
Model converged, appending LL score: -69.3590232104305
Finished Iteration: 5

Iteration: #6 : Converged after 2 iterations
Model converged, appending LL score: -69.35965661855641
Finished Iteration: 6

Iteration: #7 : Converged after 2 iterations
Model converged, appending LL score: -69.351635641764
Finished Iteration: 7

Iteration: #8 : Converged after 2 iterations
Model converged, appending LL score: -69.36333003736495
Finished Iteration: 8

Iteration: #9 : Converged after 2 iterations
Model converged, appending LL score: -69.35486670081909
Finished Iteration: 9

Now training MMM model, on full dataset with chrom #8 out.

Iteration: #0 : Converged after 2 iterations
Model converged, appending LL s

Iteration: #6 : Converged after 2 iterations
Model converged, appending LL score: -32.88253497253401
Finished Iteration: 6

Iteration: #7 : Converged after 2 iterations
Model converged, appending LL score: -32.88260599330558
Finished Iteration: 7

Iteration: #8 : Converged after 2 iterations
Model converged, appending LL score: -32.88326013570176
Finished Iteration: 8

Iteration: #9 : Converged after 2 iterations
Model converged, appending LL score: -32.875050165317425
Finished Iteration: 9

Now training MMM model, on full dataset with chrom #14 out.

Iteration: #0 : Converged after 2 iterations
Model converged, appending LL score: -46.18471994803236
Finished Iteration: 0

Iteration: #1 : Converged after 2 iterations
Model converged, appending LL score: -46.181351454512
Finished Iteration: 1

Iteration: #2 : Converged after 2 iterations
Model converged, appending LL score: -46.189010616895736
Finished Iteration: 2

Iteration: #3 : Converged after 2 iterations
Model converged, appending

Iteration: #9 : Converged after 2 iterations
Model converged, appending LL score: -19.13879480050751
Finished Iteration: 9

Now training MMM model, on full dataset with chrom #20 out.

Iteration: #0 : Converged after 2 iterations
Model converged, appending LL score: -22.739148238221194
Finished Iteration: 0

Iteration: #1 : Converged after 2 iterations
Model converged, appending LL score: -22.738887300889193
Finished Iteration: 1

Iteration: #2 : Converged after 2 iterations
Model converged, appending LL score: -22.737572617897253
Finished Iteration: 2

Iteration: #3 : Converged after 2 iterations
Model converged, appending LL score: -22.74248149917958
Finished Iteration: 3

Iteration: #4 : Converged after 2 iterations
Model converged, appending LL score: -22.740139772082507
Finished Iteration: 4

Iteration: #5 : Converged after 2 iterations
Model converged, appending LL score: -22.745733981834167
Finished Iteration: 5

Iteration: #6 : Converged after 2 iterations
Model converged, appe

In [10]:
df
df.insert(loc = 0, column = 'score', value = df.sum(axis=1))



ValueError: cannot insert score, already exists

In [16]:
df_final = df.iloc[:,0:1] #Create final DataFrame containing only the summed likelihood score 
df_final

,score
0,-1062.105507
1,-1062.097819
2,-1062.159717
3,-1062.166305
4,-1062.159072
5,-1062.116558
6,-1062.159812
7,-1062.147425
8,-1062.143529
9,-1062.161125


In [20]:
df_final['model'] = 'mmm'
df_final['iter'] = df_final.index
df_final['topics'] = 14
model_name = 'mmm'

/Users/dror/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/dror/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/dror/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [21]:
df_final.to_csv(f"data/final_data/{model_name}_crossval_results_{MMM.num_of_topics}.csv")
df_final.to_pickle(f"data/final_data/{model_name}_crossval_results_{MMM.num_of_topics}.pkl")